# Setup
This Jupyter Notebook is meant to be executed directly from Google Colab in a plug and play manner.
To enable this, the following code below can be run to set up the notebook 

In [ ]:
import os
import sys

!git clone https://github.com/terryluan12/Genrify.git
!cd Genrify && make

sys.path.insert(0, "Genrify/src")

## Preprocessing Code
The following code can be run to download the datasets and preprocess the datasets

In [1]:
from datasources import download_datasets
from preprocessing import preprocess

c:\Users\Jack\.conda\envs\aps360\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [8]:
# Make sure to change subset_num to the subset which you are using
subset_num = 3
import os
if not os.path.isdir("Genrify/src/datasources/processed_data"):
    download_datasets("Genrify/src")

preprocess(subset_num, "Genrify/src")

Downloaded Datasources
Splitting processed Data into 4 exclusive datasets.


ValueError: Sum of input lengths does not equal the length of the input dataset!

In [2]:
from cnn import mfcc_model, training
from utils import plot
from cnn.datahandler.DataHandler import DataHandler

## Example to training an model

In [1]:
# training example
# mfcc_data_dir = '~/Documents/vscode/Genrify/Data/mfcc'
# mfcc_data_handler = DataHandler(mfcc_data_dir, batch_size=32, num_workers=4)
# lr = 1e-4
# num_epochs = 30
# mfcc_cnn = mfcc_model.MFCC_CNN()
# print(f"Learning rate: {1e-4}")
# training.train(mfcc_cnn, mfcc_data_handler.train_loader, mfcc_data_handler.val_loader, num_epochs, lr, 32)
# plot.plot_training("MFCC", lr, 32, num_epochs-1)
# plot.plot_confusion_matrix("MFCC", lr, 32, num_epochs-1)